In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from scipy.spatial.distance import squareform, pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import numpy as np
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import cv2
import os
import sys
from os.path import exists
from scipy import spatial

## Settings

In [16]:
search_dir = '/home/drevital/pallet_detection/feb23_task_test_1'
clusters_base_path = '/home/drevital/pallet_detection/f1'
Path(clusters_base_path).mkdir(parents=True, exist_ok=True)
features_path = os.path.join(clusters_base_path, 'features.csv')
im_height = 200
im_width = 600
num_clusters = 5

## Similarity Function

In [17]:
def similarity(f1, f2):
    return 1 - spatial.distance.cosine(f1, f2)

## Clustering Function

In [18]:
def cluster_images(feature_vectors, similarity_function, num_clusters):
    similarities = []
    size = len(feature_vectors)
    for i in range(size):
        for j in range(i+1, size):
            similarities.append(similarity_function(feature_vectors[i], feature_vectors[j]))

    # Normalize the similarity values
    min_sim = min(similarities)
    max_sim = max(similarities)
    rng = max_sim - min_sim
    similarities = [(sim - min_sim) / rng for sim in similarities]
    
    similarities = np.array(similarities)
    pairwise_similarities = squareform(similarities)
    #clustering = AgglomerativeClustering(n_clusters=num_clusters, affinity='precomputed', linkage='average').fit(pairwise_similarities)
    clustering = AgglomerativeClustering(n_clusters=num_clusters, affinity='precomputed', linkage='complete').fit(pairwise_similarities)
    clusters = [[] for _ in range(num_clusters)]
    for i, label in enumerate(clustering.labels_):
        clusters[label].append(i)

    return clusters

## Load the MobileNet Module

In [19]:
module_handle = 'https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4'
module = hub.load(module_handle)

## Prepare compared images for features_calc and similarity search

In [20]:
sim_names = []
sim_scores = []
fnames = os.listdir(search_dir)
feature_vecs = {}
np.set_printoptions(threshold=sys.maxsize)

## Calculate and store all compared directories's images' feature-vectors

In [21]:
feature_vecs = {}
i=0

if not exists(features_path):
    for fname in tqdm(fnames):
        impath = os.path.join(search_dir, fname)
        im = tf.io.read_file(impath)
        im = tf.io.decode_jpeg(im, channels=3)
        im = tf.image.resize_with_pad(im, 224, 224)
        # Convert to shape (1, 224, 224, 3) float
        im  = tf.image.convert_image_dtype(im, tf.float32)[tf.newaxis, ...]
        f = module(im)   
        f_set = np.squeeze(f)  
        feature_vecs[i] = {'path': impath, 'features': f_set}
        i += 1
        
    features_df = pd.DataFrame.from_dict(feature_vecs).transpose() 
    features_df.to_csv(features_path, sep=',')

## Extract the Feature Vectors from the .csv File

In [22]:
features_df = pd.read_csv(features_path, delimiter=',')
feature_vectors = []
feature_vectors_map = {}

for i, fname in enumerate(fnames):
    impath = os.path.join(search_dir, fname)
    row = features_df[features_df['path'] == impath]
    fvec = [float(item) for item in row['features'].tolist()[0][1:-1].split()]
    feature_vectors.append(fvec)
    feature_vectors_map[i] = fname

## Normalize the Feature Vectors

In [23]:
#scaler = StandardScaler()
#feature_vectors = scaler.fit_transform(feature_vectors)

## Cluster Images by Feature Vectors

In [24]:
clusters = cluster_images(feature_vectors, similarity, num_clusters)
clusters_map = defaultdict(list)

for i, cluster in enumerate(clusters):
    for j in cluster:
        clusters_map[i].append(feature_vectors_map[j])
    print(f'Cluster {i+1}: {clusters_map[i]}')

Cluster 1: ['12_02_23_data_08_JP_LD.png', '12_02_23_data_04_LD.png', '12_02_23_data_07_LU.png', '13_02_23_data_00_pi.png']
Cluster 2: ['13_02_23_data_00_LU.png', '12_02_23_data_08_JP_pi.png', '12_02_23_data_04_RD.png']
Cluster 3: ['12_02_23_data_11_pi.png']
Cluster 4: ['12_02_23_data_11_LD.png']
Cluster 5: ['12_02_23_data_07_LD.png']


/home/drevital/.local/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [25]:
#similarities = [0.8985173650625983, 0.8296019645333045, 0.9272594625657268, 0.8360967426648496, 0.9417109002811195, 0.9021997480008199, 0.968168311484994, 0.8100581965109019, 0.8520442800021367, 0.8426173625200921, 0.9181628904513994, 0.8533018242091781, 0.9177115996433918, 0.969485436911368, 0.8651327513817553, 0.8296239509819362, 0.8400706374473954, 0.8828104774310855, 0.8108765722703809, 0.867970375426955, 0.8283588331795452, 0.8040750559482891, 0.9695689823703428, 0.794064387413589, 0.8632548409088351, 0.9680479669638304, 0.9037332641846758, 0.9046228872809501, 0.8786064176313063, 0.8557380785306958, 0.863823378278388, 0.8447001925475641, 0.821224416888748, 0.8190340136184544, 0.9472562816965735, 0.9147274162229626, 0.9259331304619225, 0.860495047662044, 0.8558728827763509, 0.8830891807821603, 0.8057699519359183, 0.8427360575327614, 0.7867955293572464, 0.8385347872216357, 0.7923104162621976]

#min(similarities), max(similarities)

In [26]:
#min_sim = min(similarities)
#max_sim = max(similarities)
#range = max_sim - min_sim
#similarities = [(sim - min_sim) / range for sim in similarities]
#similarities

In [27]:
feature_vectors_map

{0: '12_02_23_data_07_LD.png',
 1: '12_02_23_data_08_JP_LD.png',
 2: '13_02_23_data_00_LU.png',
 3: '12_02_23_data_11_pi.png',
 4: '12_02_23_data_04_LD.png',
 5: '12_02_23_data_11_LD.png',
 6: '12_02_23_data_08_JP_pi.png',
 7: '12_02_23_data_07_LU.png',
 8: '13_02_23_data_00_pi.png',
 9: '12_02_23_data_04_RD.png'}

In [28]:
len(feature_vectors[1])

1792